In [14]:
# ==========================================================
# SHIA ENTERPRISE HYBRID: Spark + Dask + Pandas (Código completo corregido)
# ==========================================================
# Ejecutar en Jupyter / Colab / entorno con PySpark, Dask instalados.
# Si falta alguna librería, descomenta la instalación siguiente (notebooks).
# !pip install pyspark dask distributed pandas scikit-learn joblib mlflow requests --quiet

# ----------------------------
# IMPORTS
# ----------------------------
import os
import requests
import joblib
import numpy as np
import pandas as pd
from pathlib import Path

# Spark
from pyspark.sql import SparkSession

# Dask
from dask.distributed import Client, LocalCluster

# ML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler

# Stacking / SHIA-level
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier

# MLflow
import mlflow

# Repro
RND = 42
np.random.seed(RND)

In [16]:
# ----------------------------
# 1) Descargar CSV localmente (Spark no lee https directamente)
# ----------------------------
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
local_path = "/tmp/titanic.csv"
os.makedirs("/tmp", exist_ok=True)

if not os.path.exists(local_path):
    print("Descargando dataset:", url)
    resp = requests.get(url)
    resp.raise_for_status()
    with open(local_path, "wb") as f:
        f.write(resp.content)
    print("Descarga completa ->", local_path)
else:
    print("Archivo ya existe:", local_path)

Archivo ya existe: /tmp/titanic.csv


In [17]:
# ----------------------------
# 2) Cargar con Pandas (inspección rápida)
# ----------------------------
pdf = pd.read_csv(local_path)
print("Pandas df shape:", pdf.shape)
print(pdf.head(3))

Pandas df shape: (891, 12)
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  


In [15]:
# ----------------------------
# 3) Inicializar Spark
# ----------------------------
spark = SparkSession.builder \
    .appName("SHIA_Enterprise_Hybrid") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()
print("Spark versión:", spark.version)

# Convertir Pandas -> Spark DataFrame
spark_df = spark.createDataFrame(pdf)
print("Spark DF creado:")
spark_df.show(3)


Spark versión: 3.5.1
Spark DF creado:
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|  NaN|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|  NaN|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 3 rows



In [18]:

# ----------------------------
# 4) Convertir a Dask DataFrame (opcional)
# ----------------------------
import dask.dataframe as dd
ddf = dd.from_pandas(pdf, npartitions=4)
print("Dask DataFrame particionado en:", ddf.npartitions)


Dask DataFrame particionado en: 4


In [19]:
# ----------------------------
# 5) Preprocesamiento (Spark/Pandas)
#    - Rellenar nulos, codificar sexo, seleccionar features
# ----------------------------
from pyspark.sql.functions import when, col

# Hacemos transformaciones en Spark (para simular pipeline a escala)
spark_df = spark_df.fillna({'Age': 30, 'Embarked': 'S'})
spark_df = spark_df.withColumn("Sex", when(col("Sex") == "male", 1).otherwise(0))
# Traer a Pandas para el resto del flujo (meta/SHIA)
pdf_clean = spark_df.toPandas()

features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]
target = "Survived"

X = pdf_clean[features].copy()
y = pdf_clean[target].copy()
print("Features shape:", X.shape)

Features shape: (891, 6)


In [20]:
# ----------------------------
# 6) Train/Val/Test split (Pandas)
# ----------------------------
X_train, X_hold, y_train, y_hold = train_test_split(X, y, test_size=0.30, stratify=y, random_state=RND)
X_val, X_test, y_val, y_test = train_test_split(X_hold, y_hold, test_size=0.5, stratify=y_hold, random_state=RND)
print("Train/Val/Test shapes:", X_train.shape, X_val.shape, X_test.shape)


Train/Val/Test shapes: (623, 6) (134, 6) (134, 6)


In [21]:

# ----------------------------
# 7) Iniciar Dask (LocalCluster)
# ----------------------------
cluster = LocalCluster(n_workers=4, threads_per_worker=1, memory_limit="2GB")
client = Client(cluster)
print("Dask dashboard:", client.dashboard_link)

/usr/local/lib/python3.12/dist-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35293 instead
  warnings.warn(
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:33719
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:35293/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:41557'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:44279'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:43847'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:36333'
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:46465 name: 0
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:46465
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:40864
INFO:distributed.sch

Dask dashboard: http://127.0.0.1:35293/status


In [22]:
# ----------------------------
# 8) Entrenamiento de modelos base en paralelo con Dask
#    - Función que entrena y devuelve el modelo (se recoge en el driver)
# ----------------------------
def train_model_return(name, model, X_tr, y_tr):
    """
    Entrena un modelo (pandas DataFrame/Series) y devuelve el objeto ajustado.
    """
    model.fit(X_tr, y_tr)
    return (name, model)

# modelos base
base_models = {
    "M1_rf": RandomForestClassifier(n_estimators=100, random_state=RND),
    "M2_gb": GradientBoostingClassifier(random_state=RND),
    "M3_svc": SVC(probability=True, kernel='rbf', C=2, random_state=RND),
    "M4_lr": LogisticRegression(max_iter=500, random_state=RND)
}

# enviar tareas a Dask (los datos X_train,y_train se serializan hacia los workers)
futures = []
for name, model in base_models.items():
    fut = client.submit(train_model_return, name, model, X_train, y_train)
    futures.append(fut)

# recoger resultados (modelos ajustados en driver)
trained = client.gather(futures)
models_trained = {name: mdl for name, mdl in trained}

# Guardar localmente (driver) con joblib
models_dir = Path("models")
models_dir.mkdir(exist_ok=True)
for name, mdl in models_trained.items():
    path = models_dir / f"{name}.joblib"
    joblib.dump(mdl, path)
    print("Guardado:", path)

print("Modelos base entrenados:", list(models_trained.keys()))

Guardado: models/M1_rf.joblib
Guardado: models/M2_gb.joblib
Guardado: models/M3_svc.joblib
Guardado: models/M4_lr.joblib
Modelos base entrenados: ['M1_rf', 'M2_gb', 'M3_svc', 'M4_lr']


In [23]:
# ----------------------------
# 10) Entrenar meta-learners H1, H2 y blender final Hf (Pandas/scikit-learn)
# ----------------------------
# dividir meta-features en dos para H1/H2
mid = max(1, meta_val.shape[1] // 2)
H1 = GradientBoostingClassifier(random_state=RND)
H2 = LogisticRegression(max_iter=1000, random_state=RND)

# Para entrenar H1/H2 necesitamos targets; usamos y_val aligned with meta_val rows
H1.fit(meta_val[:, :mid], y_val)
H2.fit(meta_val[:, mid:], y_val)

# features para Hf (outputs probabilísticos de H1/H2 sobre validación)
meta_for_Hf_val = np.column_stack([
    H1.predict_proba(meta_val[:, :mid])[:, 1],
    H2.predict_proba(meta_val[:, mid:])[:, 1]
])
meta_for_Hf_test = np.column_stack([
    H1.predict_proba(meta_test[:, :mid])[:, 1],
    H2.predict_proba(meta_test[:, mid:])[:, 1]
])

# Final blender Hf: StackingClassifier with H1/H2 as estimators
final_blender = StackingClassifier(
    estimators=[('h1', H1), ('h2', H2)],
    final_estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=RND),
    cv=3,
    n_jobs=1
)
final_blender.fit(meta_for_Hf_val, y_val)

# Predicción y evaluación
y_pred_test = final_blender.predict(meta_for_Hf_test)
acc = accuracy_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)
print("SHIA Final - Test Accuracy:", acc, "F1:", f1)

SHIA Final - Test Accuracy: 0.7761194029850746 F1: 0.7272727272727273


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [14:36:16] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [24]:
# ----------------------------
# 12) Guardar modelos y registrar en MLflow (local)
# ----------------------------
models_dir = Path("models")
models_dir.mkdir(exist_ok=True)

# Save base models
for name, mdl in models_trained.items():
    p = models_dir / f"{name}.joblib"
    joblib.dump(mdl, p)

# Save meta models & final
joblib.dump(H1, models_dir / "H1.joblib")
joblib.dump(H2, models_dir / "H2.joblib")
joblib.dump(final_blender, models_dir / "Hf_initial.joblib")
joblib.dump(Hf_new, models_dir / "Hf_new.joblib")
joblib.dump(Hf_final, models_dir / "Hf_final.joblib")
print("Modelos guardados en", models_dir.resolve())

# MLflow logging (local store)
mlflow.set_experiment("SHIA_Enterprise_Hybrid")
with mlflow.start_run(run_name="shia_hybrid_run"):
    mlflow.log_metric("shia_initial_acc", float(acc))
    mlflow.log_metric("shia_initial_f1", float(f1))
    mlflow.log_metric("shia_incremental_acc", float(acc_inc))
    mlflow.log_metric("shia_incremental_f1", float(f1_inc))
    # log artifacts (models)
    for file in models_dir.glob("*.joblib"):
        mlflow.log_artifact(str(file), artifact_path="models")
print("Registro MLflow completado. Revisa la carpeta ./mlruns por detalles.")

Modelos guardados en /content/models
Registro MLflow completado. Revisa la carpeta ./mlruns por detalles.


In [25]:
# ----------------------------
# 13) Cleanup recursos
# ----------------------------
client.close()
cluster.close()
spark.stop()
print("Recursos liberados. Fin del pipeline SHIA.")


INFO:distributed.scheduler:Remove client Client-fe2f31db-adc1-11f0-8179-0242ac1c000c
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:45554; closing.
INFO:distributed.scheduler:Remove client Client-fe2f31db-adc1-11f0-8179-0242ac1c000c
INFO:distributed.scheduler:Close client connection: Client-fe2f31db-adc1-11f0-8179-0242ac1c000c
INFO:distributed.scheduler:Retire worker addresses (stimulus_id='retire-workers-1760971037.4187758') (0, 1, 2, 3)
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:41557'. Reason: nanny-close
INFO:distributed.nanny:Nanny asking worker to close. Reason: nanny-close
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:44279'. Reason: nanny-close
INFO:distributed.nanny:Nanny asking worker to close. Reason: nanny-close
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:43847'. Reason: nanny-close
INFO:distributed.nanny:Nanny asking worker to close. Reason: nanny-close
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:36333'

Recursos liberados. Fin del pipeline SHIA.


# Task
Implement a machine learning pipeline that includes data augmentation, cross-validation, data architecture, and machine learning algorithm generation. The pipeline should train base models, meta-learners, and a final blender, evaluate their performance, simulate the arrival of new data for incremental training, and save the models and log metrics using MLflow.

## Generar meta-features con validación cruzada (para entrenamiento del meta-learner)

### Subtask:
Utilizar la validación cruzada en `X_train` y `y_train` con los modelos base entrenados para generar las predicciones out-of-fold (OOF). Estas predicciones OOF serán las meta-features para entrenar el meta-learner en la fase inicial.


**Reasoning**:
The subtask requires generating out-of-fold predictions using cross-validation on the training data with the trained base models. This involves iterating through the base models, splitting the training data using StratifiedKFold, training a copy of each model on the training folds, predicting on the validation folds, and storing the predictions.



In [37]:
from sklearn.model_selection import StratifiedKFold

# Initialize StratifiedKFold
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RND)

# Dictionary to store out-of-fold predictions
meta_train_oof = {}

# Iterate through base models
for name, model in models_trained.items():
    print(f"Generating OOF predictions for {name}...")
    # Array to store OOF predictions for the current model
    oof_preds = np.zeros(X_train.shape[0])

    # Iterate through cross-validation splits
    for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
        # Create a copy of the model to train on the current fold
        model_copy = model.__class__(**model.get_params())

        # Train the model copy on the training fold
        model_copy.fit(X_train.iloc[train_idx], y_train.iloc[train_idx])

        # Make predictions (probabilities for the positive class) on the validation fold
        # Check if the model has predict_proba, otherwise use predict
        if hasattr(model_copy, 'predict_proba'):
            predictions = model_copy.predict_proba(X_train.iloc[val_idx])[:, 1]
        else:
            predictions = model_copy.predict(X_train.iloc[val_idx])

        # Store predictions in the OOF array
        oof_preds[val_idx] = predictions

    # Store OOF predictions for the current model
    meta_train_oof[name] = oof_preds
    print(f"Finished OOF predictions for {name}.")

print("\nMeta-features (OOF predictions) generated:", meta_train_oof.keys())

Generating OOF predictions for M1_rf...
Finished OOF predictions for M1_rf.
Generating OOF predictions for M2_gb...
Finished OOF predictions for M2_gb.
Generating OOF predictions for M3_svc...
Finished OOF predictions for M3_svc.
Generating OOF predictions for M4_lr...
Finished OOF predictions for M4_lr.

Meta-features (OOF predictions) generated: dict_keys(['M1_rf', 'M2_gb', 'M3_svc', 'M4_lr'])


**Reasoning**:
The out-of-fold predictions for the training set have been generated. Now, the meta-features for the validation and test sets need to be generated by making predictions on these sets using the *fully trained* base models (trained on the entire training set).



In [38]:
# Generate meta-features for validation and test sets using fully trained base models
meta_val_preds = {}
meta_test_preds = {}

# Iterate through fully trained base models
for name, model in models_trained.items():
    print(f"Generating meta-features for validation and test sets using {name}...")
    # Predict probabilities on validation set
    if hasattr(model, 'predict_proba'):
        meta_val_preds[name] = model.predict_proba(X_val)[:, 1]
        meta_test_preds[name] = model.predict_proba(X_test)[:, 1]
    else:
        meta_val_preds[name] = model.predict(X_val)
        meta_test_preds[name] = model.predict(X_test)
    print(f"Finished meta-feature generation for {name}.")

print("\nMeta-features for validation generated:", meta_val_preds.keys())
print("Meta-features for test generated:", meta_test_preds.keys())

Generating meta-features for validation and test sets using M1_rf...
Finished meta-feature generation for M1_rf.
Generating meta-features for validation and test sets using M2_gb...
Finished meta-feature generation for M2_gb.
Generating meta-features for validation and test sets using M3_svc...
Finished meta-feature generation for M3_svc.
Generating meta-features for validation and test sets using M4_lr...
Finished meta-feature generation for M4_lr.

Meta-features for validation generated: dict_keys(['M1_rf', 'M2_gb', 'M3_svc', 'M4_lr'])
Meta-features for test generated: dict_keys(['M1_rf', 'M2_gb', 'M3_svc', 'M4_lr'])


**Reasoning**:
The OOF predictions for the training set (`meta_train_oof`) and the predictions for the validation and test sets (`meta_val_preds`, `meta_test_preds`) have been generated. These are currently stored as dictionaries where each key is the model name and the value is a NumPy array of predictions. To use these as meta-features for the meta-learners, they need to be stacked into NumPy arrays, where each column represents the predictions from one base model.



In [39]:
# Stack the meta-features into NumPy arrays
meta_train = np.column_stack(list(meta_train_oof.values()))
meta_val = np.column_stack(list(meta_val_preds.values()))
meta_test = np.column_stack(list(meta_test_preds.values()))

print("Shape of stacked meta_train:", meta_train.shape)
print("Shape of stacked meta_val:", meta_val.shape)
print("Shape of stacked meta_test:", meta_test.shape)

Shape of stacked meta_train: (623, 4)
Shape of stacked meta_val: (134, 4)
Shape of stacked meta_test: (134, 4)


## Entrenar los meta-learners h1 y h2

### Subtask:
Entrenar los modelos H1 y H2 utilizando las meta-features generadas a partir de la validación cruzada en el conjunto de entrenamiento (`meta_train_oof`) y el target de entrenamiento (`y_train`).


**Reasoning**:
Train the meta-learners H1 and H2 using the generated meta-features from the training set.



In [40]:
# ----------------------------
# 9) Entrenar meta-learners H1 y H2 (Pandas/scikit-learn)
#    - Usar meta-features OOF (meta_train) y targets (y_train)
# ----------------------------
# dividir meta-features en dos para H1/H2
# Ensure meta_train is a numpy array or pandas DataFrame
if isinstance(meta_train, dict):
    # If meta_train is still a dictionary of arrays, stack it
    meta_train = np.column_stack(list(meta_train.values()))

mid = max(1, meta_train.shape[1] // 2)
print(f"Dividing meta-features at column index: {mid}")

# modelos meta
H1 = GradientBoostingClassifier(random_state=RND)
H2 = LogisticRegression(max_iter=1000, random_state=RND)

# Para entrenar H1/H2 usamos las meta-features generadas con OOF (meta_train) y y_train
# Ensure y_train is a numpy array or pandas Series
if isinstance(y_train, pd.Series):
    y_train_np = y_train.values
else:
    y_train_np = y_train

print("Training H1...")
H1.fit(meta_train[:, :mid], y_train_np)
print("H1 trained.")

print("Training H2...")
H2.fit(meta_train[:, mid:], y_train_np)
print("H2 trained.")

print("Meta-learners H1 and H2 trained.")

Dividing meta-features at column index: 2
Training H1...
H1 trained.
Training H2...
H2 trained.
Meta-learners H1 and H2 trained.


## Entrenar el blender final hf (inicial)

### Subtask:
Entrenar el modelo `StackingClassifier` utilizando las predicciones probabilísticas de H1 y H2 sobre el conjunto de validación (`meta_val`) como features de entrada, y el target de validación (`y_val`).


**Reasoning**:
Instantiate and fit the StackingClassifier using the predictions of H1 and H2 on the validation set as input features and the validation target.



In [41]:
# ----------------------------
# 10) Entrenar meta-learners H1, H2 y blender final Hf (Pandas/scikit-learn)
# ----------------------------

# features for Hf (outputs probabilísticos de H1/H2 sobre validación)
# Ensure meta_val is a numpy array or pandas DataFrame
if isinstance(meta_val, dict):
    # If meta_val is still a dictionary of arrays, stack it
    meta_val = np.column_stack(list(meta_val.values()))

# Calculate the mid point based on the number of columns in meta_val
mid = max(1, meta_val.shape[1] // 2)

# Ensure y_val is a numpy array or pandas Series
if isinstance(y_val, pd.Series):
    y_val_np = y_val.values
else:
    y_val_np = y_val

# Final blender Hf: StackingClassifier with H1/H2 as estimators
final_blender = StackingClassifier(
    estimators=[('h1', H1), ('h2', H2)],
    final_estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=RND),
    cv=3,
    n_jobs=1
)

print("Training final blender (StackingClassifier)...")
# Fit the StackingClassifier to the meta_for_Hf_val (predictions of H1/H2 on validation) and y_val
# Note: The previous code block already calculated meta_for_Hf_val, but it was not stored in a variable.
# Recalculate meta_for_Hf_val using the trained H1 and H2 models on meta_val.
meta_for_Hf_val = np.column_stack([
    H1.predict_proba(meta_val[:, :mid])[:, 1],
    H2.predict_proba(meta_val[:, mid:])[:, 1]
])

final_blender.fit(meta_for_Hf_val, y_val_np)
print("Final blender (StackingClassifier) trained.")

Training final blender (StackingClassifier)...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [14:52:33] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Final blender (StackingClassifier) trained.


## Evaluar el blender final hf (inicial)

### Subtask:
Evaluar el rendimiento del `StackingClassifier` entrenado utilizando las predicciones probabilísticas de H1 y H2 sobre el conjunto de test (`meta_test`) como features de entrada, y el target de test (`y_test`).


**Reasoning**:
Evaluate the trained StackingClassifier on the test set using the predictions of H1 and H2 on meta_test as input features.



In [42]:
# Ensure meta_test is a numpy array or pandas DataFrame
if isinstance(meta_test, dict):
    # If meta_test is still a dictionary of arrays, stack it
    meta_test = np.column_stack(list(meta_test.values()))

# Calculate the mid point based on the number of columns in meta_test
mid = max(1, meta_test.shape[1] // 2)

# Generate the input features for the final blender on the test set
meta_for_Hf_test = np.column_stack([
    H1.predict_proba(meta_test[:, :mid])[:, 1],
    H2.predict_proba(meta_test[:, mid:])[:, 1]
])

# Use the trained final_blender model to make predictions on the test set meta-features
y_pred_test = final_blender.predict(meta_for_Hf_test)

# Ensure y_test is a numpy array or pandas Series
if isinstance(y_test, pd.Series):
    y_test_np = y_test.values
else:
    y_test_np = y_test

# Calculate the accuracy score
acc = accuracy_score(y_test_np, y_pred_test)

# Calculate the F1 score
f1 = f1_score(y_test_np, y_pred_test)

# Print the results
print("SHIA Final - Test Accuracy:", acc, "F1:", f1)

SHIA Final - Test Accuracy: 0.7014925373134329 F1: 0.6078431372549019


## Simular llegada de nuevos datos y generar meta-features incrementales

### Subtask:
Simulate the arrival of new data and generate the corresponding meta-features from the predictions of the trained base models.


**Reasoning**:
Generate a synthetic dataset by adding noise to a subset of the training data, create a corresponding target variable by sampling from the training target, and generate meta-features for the new data using the trained base models. Then, stack the meta-features into a NumPy array and print the shapes of the new data, target, and meta-features.



In [43]:
# ----------------------------
# 11) Simular llegada de nuevos datos (para entrenamiento incremental)
#     - Generar datos sintéticos con ruido
#     - Generar meta-features para los nuevos datos
# ----------------------------

# 1. Generate new synthetic dataset by adding random noise to a subset of the original training data
new_data_size = int(0.20 * X_train.shape[0])
X_new = X_train.sample(n=new_data_size, random_state=RND, replace=True).reset_index(drop=True)

# Calculate standard deviation for noise column-wise
std_dev = X_train.std()
noise_level = 0.01
noise = np.random.normal(0, noise_level * std_dev, size=X_new.shape)
X_new = X_new + noise
# Ensure numerical columns remain numerical after adding noise
X_new = X_new[features]

# 2. Create a corresponding target variable for the simulated data
y_new = y_train.sample(n=new_data_size, random_state=RND, replace=True).reset_index(drop=True)

# 3. Generate meta-features for the new data using the trained base models
meta_new_preds = {}
for name, model in models_trained.items():
    if hasattr(model, 'predict_proba'):
        meta_new_preds[name] = model.predict_proba(X_new)[:, 1]
    else:
        meta_new_preds[name] = model.predict(X_new)

# 4. Stack the meta-features from meta_new_preds into a NumPy array
meta_new = np.column_stack(list(meta_new_preds.values()))

# 5. Print the shapes of X_new, y_new, and meta_new
print("Shape of simulated new data (X_new):", X_new.shape)
print("Shape of simulated new target (y_new):", y_new.shape)
print("Shape of new meta-features (meta_new):", meta_new.shape)

Shape of simulated new data (X_new): (124, 6)
Shape of simulated new target (y_new): (124,)
Shape of new meta-features (meta_new): (124, 4)


## Entrenar el blender final hf (incremental)

### Subtask:
Entrenar un nuevo modelo `StackingClassifier` (`Hf_new`) utilizando los datos originales de validación (`meta_for_Hf_val`, `y_val`) y los nuevos datos simulados (`meta_new`, `y_new`) de forma incremental.


**Reasoning**:
Concatenate the original validation meta-features and target with the new simulated data and then train a new StackingClassifier incrementally.



In [44]:
# 1. Concatenate original validation meta-features and new simulated meta-features
# Ensure meta_val is a numpy array
if isinstance(meta_val, dict):
    meta_val = np.column_stack(list(meta_val.values()))

combined_meta = np.vstack([meta_val, meta_new])
print("Shape of combined meta-features:", combined_meta.shape)

# 2. Concatenate original validation target and new simulated target
# Ensure y_val and y_new are numpy arrays
if isinstance(y_val, pd.Series):
    y_val_np = y_val.values
else:
    y_val_np = y_val

if isinstance(y_new, pd.Series):
    y_new_np = y_new.values
else:
    y_new_np = y_new

combined_y = np.concatenate([y_val_np, y_new_np])
print("Shape of combined target:", combined_y.shape)


# 3. Instantiate a new StackingClassifier model (Hf_new)
# Use the same configuration as the initial blender
Hf_new = StackingClassifier(
    estimators=[('h1', H1), ('h2', H2)],
    final_estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=RND),
    cv=3,
    n_jobs=1
)

# 4. Train the new StackingClassifier (Hf_new) using the combined data
print("Training new StackingClassifier (Hf_new) incrementally...")
# The meta-features for the final blender are the probabilities from H1 and H2.
# Need to re-generate these probabilities on the combined_meta data.

mid = max(1, combined_meta.shape[1] // 2)

meta_for_Hf_combined = np.column_stack([
    H1.predict_proba(combined_meta[:, :mid])[:, 1],
    H2.predict_proba(combined_meta[:, mid:])[:, 1]
])

Hf_new.fit(meta_for_Hf_combined, combined_y)
print("New StackingClassifier (Hf_new) trained.")

Shape of combined meta-features: (258, 4)
Shape of combined target: (258,)
Training new StackingClassifier (Hf_new) incrementally...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [14:53:46] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


New StackingClassifier (Hf_new) trained.


## Evaluar el blender final hf (incremental)

### Subtask:
Evaluar el rendimiento del `StackingClassifier` entrenado incrementalmente utilizando las predicciones probabilísticas de H1 y H2 sobre el conjunto de test (`meta_test`) como features de entrada, y el target de test (`y_test`).


**Reasoning**:
Evaluate the incrementally trained StackingClassifier (Hf_new) on the test set using the specified steps.



In [45]:
# Ensure meta_test is a numpy array or pandas DataFrame
if isinstance(meta_test, dict):
    # If meta_test is still a dictionary of arrays, stack it
    meta_test = np.column_stack(list(meta_test.values()))

# Calculate the mid point based on the number of columns in meta_test
mid = max(1, meta_test.shape[1] // 2)

# Generate the input features for the incremental blender on the test set
meta_for_Hf_test = np.column_stack([
    H1.predict_proba(meta_test[:, :mid])[:, 1],
    H2.predict_proba(meta_test[:, mid:])[:, 1]
])

# Use the incrementally trained Hf_new model to make predictions on the test set meta-features
y_pred_test_inc = Hf_new.predict(meta_for_Hf_test)

# Ensure y_test is a numpy array or pandas Series
if isinstance(y_test, pd.Series):
    y_test_np = y_test.values
else:
    y_test_np = y_test

# Calculate the accuracy score of the incremental blender
acc_inc = accuracy_score(y_test_np, y_pred_test_inc)

# Calculate the F1 score of the incremental blender
f1_inc = f1_score(y_test_np, y_pred_test_inc)

# Print the calculated incremental test accuracy and F1 score
print("SHIA Incremental - Test Accuracy:", acc_inc, "F1:", f1_inc)

SHIA Incremental - Test Accuracy: 0.7313432835820896 F1: 0.660377358490566


## Guardar modelos y registrar en mlflow

### Subtask:
Guardar los modelos base, meta-learners y los diferentes blenders finales entrenados. Registrar las métricas de rendimiento de los blenders inicial e incremental en MLflow.


**Reasoning**:
Save the trained models and log metrics and artifacts using MLflow as per the instructions.



In [46]:
# ----------------------------
# 12) Guardar modelos y registrar en MLflow (local)
# ----------------------------
models_dir = Path("models")
models_dir.mkdir(exist_ok=True)

# Save base models
for name, mdl in models_trained.items():
    p = models_dir / f"{name}.joblib"
    joblib.dump(mdl, p)

# Save meta models & final
joblib.dump(H1, models_dir / "H1.joblib")
joblib.dump(H2, models_dir / "H2.joblib")
joblib.dump(final_blender, models_dir / "Hf_initial.joblib")
joblib.dump(Hf_new, models_dir / "Hf_new.joblib")
# joblib.dump(Hf_final, models_dir / "Hf_final.joblib") # Hf_final was not defined in previous steps
print("Modelos guardados en", models_dir.resolve())

# MLflow logging (local store)
mlflow.set_experiment("SHIA_Enterprise_Hybrid")
with mlflow.start_run(run_name="shia_hybrid_run"):
    mlflow.log_metric("shia_initial_acc", float(acc))
    mlflow.log_metric("shia_initial_f1", float(f1))
    mlflow.log_metric("shia_incremental_acc", float(acc_inc))
    mlflow.log_metric("shia_incremental_f1", float(f1_inc))
    # log artifacts (models)
    for file in models_dir.glob("*.joblib"):
        mlflow.log_artifact(str(file), artifact_path="models")
print("Registro MLflow completado. Revisa la carpeta ./mlruns por detalles.")

Modelos guardados en /content/models
Registro MLflow completado. Revisa la carpeta ./mlruns por detalles.


## Liberar recursos

### Subtask:
Liberar los recursos de Spark y Dask cerrando las sesiones y clusters activos.


**Reasoning**:
Close the Dask client and cluster, and stop the Spark session to release resources.



In [47]:
# ----------------------------
# 13) Cleanup recursos
# ----------------------------
client.close()
cluster.close()
spark.stop()
print("Recursos liberados. Fin del pipeline SHIA.")

INFO:distributed.scheduler:Remove client Client-edd6db4d-adc2-11f0-8179-0242ac1c000c
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:34182; closing.
INFO:distributed.scheduler:Remove client Client-edd6db4d-adc2-11f0-8179-0242ac1c000c
INFO:distributed.scheduler:Close client connection: Client-edd6db4d-adc2-11f0-8179-0242ac1c000c
INFO:distributed.scheduler:Retire worker addresses (stimulus_id='retire-workers-1760972087.8049347') (0, 1, 2, 3)
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:40155'. Reason: nanny-close
INFO:distributed.nanny:Nanny asking worker to close. Reason: nanny-close
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:38541'. Reason: nanny-close
INFO:distributed.nanny:Nanny asking worker to close. Reason: nanny-close
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:42467'. Reason: nanny-close
INFO:distributed.nanny:Nanny asking worker to close. Reason: nanny-close
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:33927'

Recursos liberados. Fin del pipeline SHIA.


Resumen:
Hallazgos Clave del Análisis de Datos
Se generaron con éxito las predicciones Out-of-Fold (OOF) para el conjunto de entrenamiento utilizando validación cruzada con los modelos base. Estas predicciones OOF sirven como las meta-características para el entrenamiento inicial del meta-aprendiz.

Se generaron las meta-características para los conjuntos de validación y prueba utilizando los modelos base completamente entrenados.

Las meta-características se apilaron en arrays de NumPy con formas (623, 4) para el entrenamiento, (134, 4) para la validación y (134, 4) para la prueba.

Se entrenaron dos meta-aprendices, H1 (Gradient Boosting Classifier) y H2 (Regresión Logística), utilizando las meta-características OOF del conjunto de entrenamiento. H1 se entrenó en la primera mitad de las meta-características y H2 en la segunda mitad.

Se entrenó un StackingClassifier inicial (blender final) utilizando las predicciones probabilísticas de H1 y H2 en el conjunto de validación como características de entrada y el objetivo de validación.

El blender final inicial logró una precisión en la prueba de aproximadamente 0.7015 y una puntuación F1 de aproximadamente 0.6078 en el conjunto de prueba.

Se generó un nuevo conjunto de datos simulado de tamaño 124 al agregar ruido a un subconjunto de los datos de entrenamiento originales, junto con una variable objetivo correspondiente.

Las meta-características para los datos nuevos simulados se generaron utilizando los modelos base entrenados, lo que resultó en un array de NumPy de forma (124, 4).

Se entrenó un nuevo StackingClassifier (Hf_new) de forma incremental combinando los datos de validación originales (meta-características y objetivo) con los datos nuevos simulados (meta-características y objetivo).

El blender final entrenado incrementalmente logró una precisión en la prueba de aproximadamente 0.7313 y una puntuación F1 de aproximadamente 0.6604 en el conjunto de prueba.

Todos los modelos entrenados (modelos base, H1, H2, blender inicial y blender incremental) se guardaron con éxito en un directorio local de "models".

Las métricas de rendimiento (precisión y puntuación F1) para el blender inicial y el incremental se registraron con éxito en MLflow, junto con los archivos de modelo guardados como artefactos.

Los recursos de Spark y Dask se liberaron con éxito cerrando las sesiones y clústeres activos.

Conclusiones
El proceso de entrenamiento incremental utilizando datos nuevos simulados resultó en una mejora tanto en la precisión como en la puntuación F1 en el conjunto de prueba en comparación con el blender inicial. Esto sugiere que la incorporación de datos nuevos puede mejorar el rendimiento del modelo.

El registro de MLflow proporciona un registro valioso del proceso de entrenamiento del modelo y las métricas de rendimiento, que puede utilizarse para el seguimiento de experimentos, la comparación de diferentes versiones de modelos y la reproducción de resultados.